In [1]:
!pip install -Uq fastai
!pip install -Uq "timm==0.6.2.dev0"

from google.colab import drive, runtime
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
### Importing common items
import sys
import numpy as np
import pandas as pd
from PIL import Image

from fastai.vision.all import *
import timm

import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')
plt.style.use('seaborn-v0_8-colorblind')
plt.rcParams['axes.grid'] = False

In [3]:
base_directory = '/content/drive/MyDrive/ThesisGD/'
images_by_name_file = base_directory + 'images_by_name.pkl'
images_by_oid_file = base_directory + 'images_by_oid.pkl'

sys.path.insert(0, base_directory)

In [4]:
# base_directory = '/home/shoaib/ZTFDataChallenge/'
# images_by_name_file = base_directory + 'GoogleDrive/images_by_name.pkl'
# images_by_oid_file = base_directory + 'GoogleDrive/images_by_oid.pkl'

# sys.path.insert(0, base_directory + 'GoogleDrive/')

In [5]:
from fastai_catagory_run import *

In [6]:
### Loading the images by name
df = pd.read_pickle(images_by_name_file)

# ### Loading the lightcurves by OID
# df = pd.read_pickle(images_by_oid_file)

In [7]:
df = df.dropna(axis=0)

df = df.query("r_n_good_det_1day >= 20 and g_n_good_det_1day >= 20")
# df = df.query("n_good_det_1day >= 20")

catagories = ['Q', 'QR', 'QX']
# catagories = ['Q', 'QX']
# catagories = df.type.value_counts().index[:6]
df = df.query("type in @catagories")
df

,name,type,z,X,r_n_good_det,g_n_good_det,X_1day,r_n_good_det_1day,g_n_good_det_1day
42,1WGA J0852.8+1346,QX,1.49,<PIL.Image.Image image mode=RGB size=50x50>,449.0,121.0,<PIL.Image.Image image mode=RGB size=50x50>,140.0,113.0
43,1WGA J0924.4+5215,QX,0.73,<PIL.Image.Image image mode=RGB size=50x50>,408.0,161.0,<PIL.Image.Image image mode=RGB size=50x50>,208.0,146.0
45,1WGA J1008.6+5437,QX,0.78,<PIL.Image.Image image mode=RGB size=50x50>,531.0,316.0,<PIL.Image.Image image mode=RGB size=50x50>,244.0,245.0
46,1WGA J1009.4+5334,QX,1.73,<PIL.Image.Image image mode=RGB size=50x50>,501.0,288.0,<PIL.Image.Image image mode=RGB size=50x50>,236.0,224.0
48,1WGA J1052.6+5724,QX,1.11,<PIL.Image.Image image mode=RGB size=50x50>,812.0,614.0,<PIL.Image.Image image mode=RGB size=50x50>,312.0,317.0
...,...,...,...,...,...,...,...,...,...
30246,XMDS J02234-0454,QX,0.60,<PIL.Image.Image image mode=RGB size=50x50>,169.0,152.0,<PIL.Image.Image image mode=RGB size=50x50>,142.0,131.0
30249,XMS J103020.7+305637,QX,1.07,<PIL.Image.Image image mode=RGB size=50x50>,274.0,229.0,<PIL.Image.Image image mode=RGB size=50x50>,200.0,188.0
30250,XMS J123139.9+641123,QX,1.02,<PIL.Image.Image image mode=RGB size=50x50>,784.0,713.0,<PIL.Image.Image image mode=RGB size=50x50>,335.0,307.0
30256,ZC 2357-027,Q,2.61,<PIL.Image.Image image mode=RGB size=50x50>,172.0,155.0,<PIL.Image.Image image mode=RGB size=50x50>,136.0,130.0


### Deciding which models to use

In [8]:
timm.list_models('swin*', pretrained=True)

['swin_base_patch4_window7_224.ms_in1k',
 'swin_base_patch4_window7_224.ms_in22k',
 'swin_base_patch4_window7_224.ms_in22k_ft_in1k',
 'swin_base_patch4_window12_384.ms_in1k',
 'swin_base_patch4_window12_384.ms_in22k',
 'swin_base_patch4_window12_384.ms_in22k_ft_in1k',
 'swin_large_patch4_window7_224.ms_in22k',
 'swin_large_patch4_window7_224.ms_in22k_ft_in1k',
 'swin_large_patch4_window12_384.ms_in22k',
 'swin_large_patch4_window12_384.ms_in22k_ft_in1k',
 'swin_s3_base_224.ms_in1k',
 'swin_s3_small_224.ms_in1k',
 'swin_s3_tiny_224.ms_in1k',
 'swin_small_patch4_window7_224.ms_in1k',
 'swin_small_patch4_window7_224.ms_in22k',
 'swin_small_patch4_window7_224.ms_in22k_ft_in1k',
 'swin_tiny_patch4_window7_224.ms_in1k',
 'swin_tiny_patch4_window7_224.ms_in22k',
 'swin_tiny_patch4_window7_224.ms_in22k_ft_in1k',
 'swinv2_base_window8_256.ms_in1k',
 'swinv2_base_window12_192.ms_in22k',
 'swinv2_base_window12to16_192to256.ms_in22k_ft_in1k',
 'swinv2_base_window12to24_192to384.ms_in22k_ft_in1k',


In [9]:
models = ['resnet18',
          'resnet50',
          'densenet121',
          'convnext_small',
          'convnext_base',
          'resnext50_32x4d',
          'efficientvit_b0',
          'efficientvit_m5',
          'efficientnetv2_rw_m',
          'vit_base_mci_224',
          'swin_base_patch4_window7_224',
        ]

pass_if_nonempty = [False for model in models]
batch_sizes = [64 for model in models]

save_dirs = [base_directory + 'LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/'+ model + '/' for model in models]
save_dirs

['/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/resnet18/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/resnet50/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/densenet121/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/convnext_small/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/convnext_base/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/resnext50_32x4d/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/efficientvit_b0/',
 '/content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_UnweightedCrossEntropLoss_3QSOClasses/efficien

In [ ]:
def run_a_model(**kwargs):
    model_run = FastaiRun(**kwargs)
    try:
        model_run.start_learning()
    except RuntimeError as e:
        if batch_size <= 8:
            raise RuntimeError(f'Model {kwargs['cnn_model']} could not run because of insufficient GPU memory.')
        else:
            print(str(e) + f'\n\nNow trying with batch_size {batch_size / 2}')
            new_kwargs = kwargs.copy()
            new_kwargs['batch_size'] = batch_size / 2
            run_a_model(**kwargs)
    model_run.save_classification_results()


for model, save_dir, pass_run, batch_size in zip(models, save_dirs, pass_if_nonempty, batch_sizes):
    try:
        run_a_model(save_directory=save_dir,
                    image_df=df,
                    image_df_Xcol='X_1day',
                    image_df_ycol='type',
                    pass_if_nonempty_dir=pass_run,
                    overwrite_save_directory=False,
                    cnn_model=model,
                    batch_size=batch_size)
    except Exception as e:
        print(model + ' did not run due to the exception:\n' + str(e))

resnet18 did not run due to the exception:
Save directory is not empty and the object will pass it: /content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_WeightedCrossEntropLoss_3QSOClasses/resnet18/
resnet50 did not run due to the exception:
Save directory is not empty and the object will pass it: /content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_WeightedCrossEntropLoss_3QSOClasses/resnet50/
densenet121 did not run due to the exception:
Save directory is not empty and the object will pass it: /content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_WeightedCrossEntropLoss_3QSOClasses/densenet121/
convnext_small did not run due to the exception:
Save directory is not empty and the object will pass it: /content/drive/MyDrive/ThesisGD/LCsByName_ImbalancedTraining_F1Metric_WeightedCrossEntropLoss_3QSOClasses/convnext_small/
convnext_base did not run due to the exception:
Save directory is not empty and the object will pass it: /content/driv

/usr/local/lib/python3.11/dist-packages/fastai/learner.py:67: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


vit_base_mci_224 did not run due to the exception:
Unknown model (vit_base_mci_224)


/usr/local/lib/python3.11/dist-packages/fastai/learner.py:67: UserWarning: Saved file doesn't contain an optimizer state.
  elif with_opt: warn("Saved file doesn't contain an optimizer state.")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
runtime.unassign()